Load data, split into train and test in two folders then adapt a tolkenizer and save result on train data.

Adapt a tokenizer on train data and save its token list in a file (one token per line). This will be used to tokenise text in clip model training and inference

In [59]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [60]:
import sys
my_local_drive='/content/gdrive/MyDrive/ML2_projet'
sys.path.append(my_local_drive)

In [61]:
%cd $my_local_drive

/content/gdrive/MyDrive/ML2_projet


In [62]:
import os
import pandas as pd
import re
import numpy as np
import random
import zipfile
import requests
import io
import math
from pathlib import Path
import shutil
import datetime

from dataclasses import dataclass

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from tensorflow.keras.metrics import Mean
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Pour utiliser au mieux le GPU
AUTOTUNE = tf.data.AUTOTUNE

In [63]:
def copy_dateset(df, base_dir, target_dir):
  # copy a dataset into new datasets
  for index, row in df.iterrows():
    # very slow and ineficient but only ran once
    base_image_path = os.path.join(base_dir, row["image_path"])
    copy_image_path = os.path.join(target_dir, row["image_path"])

    text_id = row["image_path"].split('/')[-1] # text is not subsplit in folders like images.
    text_id = text_id.replace(".jpg", ".txt")  # change extension
    base_text_path = os.path.join(base_dir, "captions", text_id)
    copy_text_path = os.path.join(target_dir, "captions", text_id)

    print(f"moving file : {base_image_path}, {copy_image_path}")
    shutil.copy(base_image_path, copy_image_path)
    print(f"moving file : {base_text_path}, {copy_text_path}")
    shutil.copy(base_text_path, copy_text_path)

In [75]:
# Répertoire cible pour sauvegarder vos modèles
model_dir = "./models_forclip"
os.makedirs(model_dir, exist_ok=True)

# Répertoire des données
dataset_dir = "./flickr_subset2"

# storing tokenizer vocab noe token per line
vocab_path = os.path.join(dataset_dir, "vocab.txt")


# Répertoire des images
image_dir = os.path.join(dataset_dir, "images")
# Répertoire des captions
captions_dir = os.path.join(dataset_dir, "captions")

# Variables utiles
# Attention respecter bien l'ordre alphabétique des classes pour
# le générateur
class_names = ['ball', 'bike', 'dog', 'water']
# class encoding dict
class_dict = {
    "ball": 0,
    "bike": 1,
    "dog": 2,
    "water": 3
}


# train
train_dir = os.path.join(dataset_dir, "train_data")
train_image_dir = os.path.join(train_dir, "images")
train_captions_dir = os.path.join(train_dir, "captions")

# test
test_dir = os.path.join(dataset_dir, "test_data")
test_image_dir = os.path.join(test_dir, "images")
test_captions_dir = os.path.join(test_dir, "captions")

# create directories
for dir in [train_dir, train_image_dir, train_captions_dir, test_dir, test_image_dir, test_captions_dir]:
    os.makedirs(dir, exist_ok=True)

# create subdirectories for image classes
for class_name in class_names:
    os.makedirs(os.path.join(train_image_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_image_dir, class_name), exist_ok=True)


# Pour les images
image_size=(224, 224)
image_shape = image_size + (3,)


# Pour les textes
sequence_length = 32
vocab_size = 10000
num_heads = 4
ff_dim = 256
num_layers = 2

# Pour les images et les textes dans le modèle CLIP
embed_dim = 128

# pour le training:
batch_size = 16

In [65]:
caption_file_path = Path(dataset_dir) / "captions.csv"
captions_df = pd.read_csv(caption_file_path, sep=",")
captions_df.head()

train_df, test_df = train_test_split(
    captions_df,
    test_size=0.2,
    random_state=42,
    stratify=captions_df["label"])

# copy data files
copy_dateset(train_df, dataset_dir, train_dir)
copy_dateset(test_df, dataset_dir, test_dir)

# create new captions csv
train_df.to_csv(os.path.join(train_dir, "captions.csv"), quoting=0, index=False)
test_df.to_csv(os.path.join(test_dir, "captions.csv"), quoting=0, index=False)


moving file : ./flickr_subset2/images/ball/ball_029.jpg, ./flickr_subset2/train_data/images/ball/ball_029.jpg
moving file : ./flickr_subset2/captions/ball_029.txt, ./flickr_subset2/train_data/captions/ball_029.txt
moving file : ./flickr_subset2/images/water/water_090.jpg, ./flickr_subset2/train_data/images/water/water_090.jpg
moving file : ./flickr_subset2/captions/water_090.txt, ./flickr_subset2/train_data/captions/water_090.txt
moving file : ./flickr_subset2/images/ball/ball_057.jpg, ./flickr_subset2/train_data/images/ball/ball_057.jpg
moving file : ./flickr_subset2/captions/ball_057.txt, ./flickr_subset2/train_data/captions/ball_057.txt
moving file : ./flickr_subset2/images/bike/bike_095.jpg, ./flickr_subset2/train_data/images/bike/bike_095.jpg
moving file : ./flickr_subset2/captions/bike_095.txt, ./flickr_subset2/train_data/captions/bike_095.txt
moving file : ./flickr_subset2/images/ball/ball_106.jpg, ./flickr_subset2/train_data/images/ball/ball_106.jpg
moving file : ./flickr_subse

TypeError: join() got an unexpected keyword argument 'sep'

# Train tokenizer

In [71]:
train_df.head()

,image_path,label,caption
328,images/ball/ball_029.jpg,ball,A brown dog is jumping after a tennis ball .
239,images/water/water_090.jpg,water,A jacketed man with his back to the camera is ...
356,images/ball/ball_057.jpg,ball,A boy does a cannonball into a pool while a gi...
544,images/bike/bike_095.jpg,bike,A man on a bicycle rides on a mountain .
405,images/ball/ball_106.jpg,ball,A little boy kicks a soccer ball in the park .


In [77]:
tokenizer = TextVectorization(
        max_tokens=vocab_size,
        standardize='lower_and_strip_punctuation',
        split='whitespace',
        pad_to_max_tokens=True,
        output_sequence_length=sequence_length,
        output_mode="int"  # save 0 for pad tokens
      )

tokenizer.adapt(train_df['caption'])
vocab = tokenizer.get_vocabulary()
print(vocab)

# save vocab
with open(vocab_path, 'w') as f:
    for token in vocab[2:]:
        f.write(token + '\n')




['', '[UNK]', np.str_('a'), np.str_('in'), np.str_('the'), np.str_('dog'), np.str_('on'), np.str_('and'), np.str_('is'), np.str_('black'), np.str_('man'), np.str_('with'), np.str_('of'), np.str_('boy'), np.str_('water'), np.str_('white'), np.str_('brown'), np.str_('ball'), np.str_('his'), np.str_('bike'), np.str_('to'), np.str_('red'), np.str_('girl'), np.str_('riding'), np.str_('jumping'), np.str_('blue'), np.str_('while'), np.str_('football'), np.str_('wearing'), np.str_('running'), np.str_('are'), np.str_('two'), np.str_('at'), np.str_('an'), np.str_('through'), np.str_('grass'), np.str_('down'), np.str_('bicycle'), np.str_('shirt'), np.str_('into'), np.str_('by'), np.str_('playing'), np.str_('group'), np.str_('dogs'), np.str_('standing'), np.str_('runs'), np.str_('mouth'), np.str_('from'), np.str_('field'), np.str_('yellow'), np.str_('another'), np.str_('air'), np.str_('near'), np.str_('child'), np.str_('rides'), np.str_('plays'), np.str_('people'), np.str_('motorcycle'), np.str_('

In [78]:
plot_history_simple(history)

NameError: name 'plot_history_simple' is not defined